# Decorators and Object-Oriented Programming


## The `@property` decorator

In some languages, it is _strongly discouraged_ to make any data attributes public, as we do here with a `Duck` class:

In [1]:
class DuckV1:
    def __init__(self, name):
        self.name = name
        
d1 = DuckV1('Donald')
d1.name

'Donald'

Let's say we have some code that _uses_ the `Duck` class:

In [2]:
def talk_like_a(duck, message):
    print(f'{duck.name} the Duck says "{message}"')
    
talk_like_a(d1, 'Hey there, Daisy!')

Donald the Duck says "Hey there, Daisy!"


Now let's imagine that we have allowed users to create their own `Duck`s, with the ability to update the duck's name. Unfortunately, our target audience is using "inappropriate" name, and, and we want to ensure that the duck names are "appropriate," so we replace the public attribute with a "getter" and a "setter":

In [3]:
class DuckV2:
    def __init__(self, name):
        self._name = name
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        if value.lower() in ('porky', 'ed', 'bugs'):
            raise ValueError('Inappropriate name!')
        self._name = value
    
        
d2 = DuckV2('Donald')
d2.get_name()

'Donald'

In [4]:
d2.set_name('Ed')

ValueError: Inappropriate name!

The problem is that now, we've broken our `talk_like_a` function:

In [5]:
talk_like_a(d2, "Hey there, Daisy!")

AttributeError: 'DuckV2' object has no attribute 'name'

(and that's why some languages say to _start_ with getters and setters)

Python, however, has a solution for this: the `property` function creates a special "descriptor" object that allows us to use getters and setters that are accessed _just like attributes_:

In [6]:
class DuckV3:
    def __init__(self, name):
        # self.name = name
        # self.set_name(name)
        self._name = name
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        if value.lower() in ('porky', 'ed', 'bugs'):
            raise ValueError('Inappropriate name!')
        self._name = value
        
    name = property(get_name, set_name)
    
        
d3 = DuckV3('Donald')
d3.name

'Donald'

In [7]:
d3.name = 'Ed'

ValueError: Inappropriate name!

... and it works with our function now:

In [8]:
talk_like_a(d3, "Hey there, Daisy!")

Donald the Duck says "Hey there, Daisy!"


### The decorator syntax

The code above is *ok*, but could be improved:

- We don't need access to the old `get_name` and `set_name` functions any more since we can use them via the `property`
- The fact that the getter and setter are later wrapped up in a property is kind of obscured by being lower in the class definition.

To rectify this, we can use the decorator syntax:

If we type:

```python
@foo
def my_function():
    ...
```

then Python "re-writes" this as:

```python
def my_function():
    ...
my_function = foo(my_function)
```

In [9]:
class DuckRO:
    def __init__(self, name):
        self._name = name
        
#     def name(self):
#         return self._name
#     name = property(name)
    
    @property
    def name(self):
        return self._name
    
        
d3a = DuckRO('Donald')
d3a.name

'Donald'

In [10]:
d3a.name = 'Hewey'

AttributeError: can't set attribute

### The @property decorator

The `property` function can be used as a decorator, as well, with the following syntax:

In [11]:
class DuckV4(object):
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self._name
    # name is a property object on this line
    
    @name.setter  # also name.deleter for `del duck.name`
    def name(self, val):
        '''setter for name attribute'''
        if val.lower() in ('porky', 'ed', 'bugs'):
            raise ValueError('Inappropriate name!')
        self._name = val
    
#     name = name.setter(name)

In [12]:
class DuckV4(object):
    def __init__(self, name):
        self._name = name
        
    def name(self):
        '''getter for name attribute'''
        return self._name
    # name is a function
    name = property(name)
    # name is a property
    
    _tmp_deco = name.setter
    def name(self, val):
        '''setter for name attribute'''
        if val.lower() in ('porky', 'ed', 'bugs'):
            raise ValueError('Inappropriate name!')
        self._name = val
    # name is a function (again)
    name = _tmp_deco(name)
    # name is a property (again)
    

In [13]:
d4 = DuckV4('Donald')
d4.name

'Donald'

In [14]:
d4.name = 'Daffy'
d4.name

'Daffy'

In [15]:
d4.name = 'Bugs'

ValueError: Inappropriate name!

In [16]:
d4.__dict__

{'_name': 'Daffy'}

In [17]:
talk_like_a(d4, 'Hey there, Daisy!')

Daffy the Duck says "Hey there, Daisy!"


How does this work with inheritance?

In [ ]:
class Mallard(DuckV4):
    @property
    def name(self):
        return super().name
    
    @name.setter
    def name(self, value):
        if value.lower().startswith('d'):
            raise ValueError
        super().name =  value  #? broken

In [ ]:
d5 = Mallard('Hewey')
d5.name

In [ ]:
d5.name = 'Donald'

In [ ]:
d5.name = 'Porky'

In [ ]:
d5.name = 'Howard'

Practically speaking, we can also use `@property` to create computed attributes:

In [ ]:
class Rect:
    
    def __init__(self, w, h):
        self.w, self.h = w, h
        
    @property
    def area(self):
        return self.w * self.h
        
r = Rect(3,4)

In [ ]:
r.area

In [ ]:
r.w = 4
r.area

In [ ]:
r.area = 5

We can even enhance the above by *caching* our computed attribute

In [ ]:
class Rect(object):
    
    def __init__(self, w, h):
        self._w, self._h = w, h
        self._area = None
        
    @property
    def w(self):
        return self._w
    @w.setter
    def w(self, value):
        self._w = value
        self._area = None
        
    @property
    def h(self):
        return self._h
    @h.setter
    def h(self, value):
        self._h = value
        self._area = None
        
    @property
    def area(self):
        print('get area')
        if self._area is None:
            self._area = self._calc_area()
        return self._area
        
    def _calc_area(self):
        print('calc area')
        return self._w * self._h
    
r = Rect(3,4)


In [ ]:
r.area

In [ ]:
r.__dict__

In [ ]:
r.area

In [ ]:
r.w = 4

In [ ]:
r.__dict__

In [ ]:
r.area

In [ ]:
r.area = 5

## Class attributes vs. Instance attributes
* attributes set outside `__init__` belong to the *class* (as opposed to the *instance*) and are shared by all instances of the class
    * these attributes can be accessed via __`ClassName.var`__ and __`classInstance.var`__
* attributes created inside `__init__` (and all other method functions) and prefaced with __`self.`__ belong to the object *instance* and cannot be accessed via __`ClassName.`__

In [ ]:
class Person:
    species = 'Human'
    
    def __init__(self, name):
        self.name = name
        print(
            "{}'s species is {}".format(
            self.name, self.species))

In [ ]:
p1 = Person('Godzilla')

In [ ]:
Person.species, p1.species, p1.name

In [ ]:
Person.name

http://www.pythontutor.com/

In [ ]:
p2 = Person('Mothra')
p2.name, p2.species

In [ ]:
Person.species = 'animal'

In [ ]:
p1.species, p2.species, Person.species

In [ ]:
p1.species = 'monster'

In [ ]:
Person.species, p1.species, p2.species

# Static and Class Methods
* static methods are methods that don't operate on an instance of the object and therefore are shared by all instances of the object
* class methods are methods that operate on the class itself, rather than instance of the class

In [ ]:
class Duck:
    family = 'Bird'
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self._name

    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        self._name = val
    
    @staticmethod
    def myprint(thing):
        '''note that self is NOT the first param'''
        msg = f'{Duck.family}: {thing}'
        print('-' * len(msg))
        print(msg)
        print('-' * len(msg))

In [ ]:
d = Duck('Donald')
d.myprint('Some thing')

In [ ]:
Duck.myprint('Some thing via the class')

In [ ]:
class Duck:
    family = 'Bird'
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self._name

    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        self._name = val
    
    @staticmethod
    def myprint(thing):
        '''note that self is NOT the first param'''
        msg = f'{Duck.family}: {thing}'
        print('-' * len(msg))
        print(msg)
        print('-' * len(msg)) 
        
    @classmethod
    def myprint_cls(cls, thing):
        '''note that self is NOT the first param'''
        msg = f'{cls.family}: {thing}'
        print('-' * len(msg))
        print(msg)
        print('-' * len(msg))
        
    #myprint = staticmethod(myprint)

In [ ]:
d = Duck('Donald')
d.myprint_cls('Some thing')
d.myprint('Some thing')

In [ ]:
class Mallard(Duck): 
    family = 'Avia'


In [ ]:
d = Mallard('Daffy')

In [ ]:
d.myprint('Some thing')   # staticmethod

In [ ]:
d.myprint_cls('Some thing')

### class/static methods as "virtual constructors"

Suppose we want to let users create Ducks with a randomly-selected name. 

We *could* do this with a @staticmethod as follows:

In [ ]:
import random

class Duck:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self._name

    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        self._name = val
        
    @staticmethod
    def with_random_name(options):
        name = random.choice(options)
        result = Duck(name)
        return result

In [ ]:
d = Duck.with_random_name(['Daffy', 'Donald', 'Scrooge'])
d.name

This breaks down with our Mallard class, though...

In [ ]:
class Mallard(Duck): pass

In [ ]:
d = Mallard.with_random_name(['Daffy', 'Donald', 'Scrooge'])
print(d.name, 'is a', type(d))

We can fix this by making our method a @classmethod:

In [ ]:
import random

class Duck(object):
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self._name

    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        self._name = val
        
    @classmethod
    def with_random_name(cls, options):
        name = random.choice(options)
        result = cls(name)
        return result

In [ ]:
d = Duck.with_random_name(['Daffy', 'Donald', 'Scrooge'])
d.name, type(d)

In [ ]:
class Mallard(Duck): pass

In [ ]:
d = Mallard.with_random_name(['Daffy', 'Donald', 'Scrooge'])
print(d.name, 'is a', type(d))

### class methods for accessing  class data

Although we can always "hard-code" the class name when accessing class data attributes directly, it's less fragile to use a `@classmethod`:

In [ ]:
import random

class Duck:
    # Key is the name of the duck, value is the Duck
    _ducks = {}
    
    def __init__(self, name):
        self._name = name
        self._ducks[name] = self
        # self._ducks.__setitem__(name, self)
        
    def __repr__(self):
        return f'<{type(self).__name__}: {self._name}>'
        
    @property
    def name(self):
        '''getter for name attribute'''
        return self._name

    @name.setter
    def name(self, val):
        '''setter for name attribute'''
        self._name = val
        
    @classmethod
    def by_name(cls, name):
        return cls._ducks[name]

In [ ]:
Duck('Hewey')
Duck('Dewey')
Duck("Lewey");

In [ ]:
Duck.by_name('Hewey')

In [ ]:
Duck.by_name('Dewey')

Now if we have a `Mallard` class, we can choose to either maintain all the Mallards in the `Duck` registry or create their own registry:

In [ ]:
class SharedRegistryMallard(Duck): pass

class PrivateRegistryWoodDuck(Duck):
    _ducks = {}  # Shadows the Duck._ducks class attribute

In [ ]:
SharedRegistryMallard('Gus')
SharedRegistryMallard('Kishore')
PrivateRegistryWoodDuck('Horatio')
PrivateRegistryWoodDuck('Max');

In [ ]:
Duck._ducks

In [ ]:
PrivateRegistryWoodDuck._ducks

In [ ]:
Duck.by_name('Gus')

In [ ]:
PrivateRegistryWoodDuck.by_name('Horatio')

## Summary of instance, static, and class methods

In [ ]:
class Foo:
    def __init__(self, name):
        self._name = name
    
    def __repr__(self):
        return f'<{self._name}>'
    
    def method(*args):
        print(f'method{args}')
        
    @staticmethod
    def static(*args):
        print(f'static{args}')
        
    @classmethod
    def clsmethod(*args):
        print(f'classmethod{args}')

In [ ]:
bar = Foo('bar')

In [ ]:
Foo.method(1, 2)  # e.g. str.startswith(s, 'Foo')

In [ ]:
bar.method(1, 2)

In [ ]:
Foo.static(1, 2)

In [ ]:
bar.static(1, 2)

In [ ]:
Foo.clsmethod(1, 2)

In [ ]:
bar.clsmethod(1, 2)

# Lab

Open the [OOP Decorators Lab][oop-decorators-lab]

[oop-decorators-lab]: ./oop-decorators-lab.ipynb